In [2]:
pip install bs4


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install html5lib


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import time
import requests
import re
import lxml

In [5]:

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
#driver = webdriver.Chrome()
#driver.get(URL)
#time.sleep(3)
#html = driver.page_source
#soup = BeautifulSoup(html, 'html.parser')
#r = requests.get(url=URL, headers=headers)
#driver.switch_to.frame("sp_message_iframe_764224") 
#element = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[3]/div[2]/button').click()

In [5]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
headers = {'User-Agent': USER_AGENT}

In [6]:
league_codes = ["AN1L", "AR1N", "AUS1", "A1", "BGD1", "BE1", "BO1A", "BRA1", "CDN1", "CLPD", "CSL", "COLP", "COL1", "CRPD", "PDV1", "KR1", "TS1",
"DK1", 
"EL1A", 
"EL1S", 
"SL1A", 
"GB1", 
"GB2", 
"GB3", 
"GB4", 
"FIJ1", 
"FR1", 
"FR2", 
"L1", 
"L2", 
"L3", 
"GHPL", 
"GR1", 
"GU1A", 
"GU1C", 
"HGKG", 
"UNG1", 
"IND1", 
"IN1L", 
"IRN1", 
"IR1", 
"ISR1", 
"IT1", 
"IT2", 
"JPL1", 
"JAP1", 
"RSK1", 
"KG1L", 
"MYS1", 
"MEXA", 
"MARP", 
"MO1L", 
"MYA1", 
"NPL1", 
"NL1", 
"NCL1", 
"NNL1", 
"NO1", 
"PN1A", 
"PN1C", 
"PR1A", 
"PR1C", 
"TDeA", 
"TDeC", 
"PFL1", 
"PL1", 
"PO1", 
"QSL", 
"RO1", 
"RU1", 
"SA1", 
"SC1", 
"SER1", 
"SIN1", 
"SFA1", 
"ES1", 
"ES2", 
"SE1", 
"C1", 
"TAD1", 
"THA1", 
"TUN1", 
"TUSC", 
"TR1", 
"UKR1", 
"UAE1", 
"MLS1", 
"URU1", 
"UZ1", 
"VZ1L", 
"VIE1"]
list_seasons = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

In [7]:
url_temp = "https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league}/plus/?saison_id={season}"


In [8]:
list_urls = []
for season in list_seasons:
    for league in league_codes:
        url = url_temp.format(league=league, season=season)
        print(url)
        r = requests.get(url=url, headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find("table", class_='items')

        if table is not None:
            tbody = table.find("tbody")
            rows = tbody.find_all("tr")
            for row in rows:
                team_links = row.select('a')[0]['href']
                list_urls.append("https://www.transfermarkt.co.uk" + team_links)

        else:
            print('No teams found')

https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/AN1L/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/AR1N/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/AUS1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/A1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/BGD1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/BE1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/BO1A/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/BRA1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/CDN1/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/CLPD/plus/?saison_id=2016
https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/CSL/pl

KeyboardInterrupt: 

In [10]:
len(list_urls)

9764

In [11]:
list_player_url = set()
for link in list_urls:
    print(link)
    r = requests.get(url=link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table", class_='items')

    if table is not None:
        tbody = table.find("tbody")
        rows = tbody.find_all("tr")
        for row in rows:
            cells = row.find_all('td', 'hauptlink')
            if cells:
                player_link = cells[0].find('a')['href']
                list_player_url.add("https://www.transfermarkt.co.uk" + player_link)
    else:
        print("Table with class 'items' not found on the page.")

list_player_url = list(list_player_url)


https://www.transfermarkt.co.uk/ca-petroleos-luanda/startseite/verein/7084/saison_id/2020
https://www.transfermarkt.co.uk/cd-1-ordm-de-agosto/startseite/verein/14503/saison_id/2020
https://www.transfermarkt.co.uk/fc-onze-bravos-do-maquis/startseite/verein/35599/saison_id/2020
https://www.transfermarkt.co.uk/gd-interclube-luanda/startseite/verein/9539/saison_id/2020
https://www.transfermarkt.co.uk/grupo-desportivo-sagrada-esperanca/startseite/verein/26741/saison_id/2020
https://www.transfermarkt.co.uk/desportivo-da-huila/startseite/verein/39419/saison_id/2020
https://www.transfermarkt.co.uk/recreativo-do-libolo/startseite/verein/26740/saison_id/2020
https://www.transfermarkt.co.uk/academica-petroleos-do-lobito/startseite/verein/46565/saison_id/2020
https://www.transfermarkt.co.uk/progresso-do-sambizanga/startseite/verein/35308/saison_id/2020
https://www.transfermarkt.co.uk/sporting-clube-petroleos-de-cabinda/startseite/verein/36832/saison_id/2020
https://www.transfermarkt.co.uk/clube-re

KeyboardInterrupt: 

In [12]:
len(list_player_url)

17265

In [13]:
def create_dict(player_name, DOB, POB, age, height, citizenship, position, foot, agent, current_club, joined, contract_expiry):

    player_info = {
        "Player Name": player_name,
        "Date of Birth": DOB,
        "Place of Birth": POB,
        "Age": age,
        "Height": height,
        "Citizenship": citizenship,
        "Position": position,
        "Foot": foot,
        "Agent": agent,
        "Current Club": current_club,
        "Joined": joined,
        "Contract Expiry": contract_expiry
    }
    return player_info




In [32]:
player_bio_list = []

while len(player_bio_list) < 2:
    for player in list_player_url:
        print(player)
        r = requests.get(url=player, headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('div', class_='info-table')
        print(table.prettify())

    for row in table:
        player_name = soup.find(text='name:').next('span') if player_name else None
        print(player_name)

        DOB = soup.find(text='Date of birth:').parent.parent.findNext('td').contents[0] if DOB else None
        print(DOB)

        POB = soup.find(text='Place of birth:').parent.parent.findNext('td').contents[0] if POB else None
        print(POB)
        
        age = soup.find(text='Age:').parent.parent.findNext('td').contents[0] if age else None
        print(age)
        
        height = soup.find(text='Height:').parent.parent.findNext('td').contents[0] if height else None
        print(height)
        
        citizenship = soup.find(text='Citizenship').parent.parent.findNext('td').contents[0] if citizenship else None
        print(citizenship)
        
        position = soup.find(text='Position').parent.parent.findNext('td').contents[0] if position else None
        print(position)
        
        foot = soup.find_next('span', class_='info-table__content--bold').parent.parent.findNext('td').contents[0] if foot else None
        print(foot)
        
        agent = soup.find_next('span', class_='info-table__content--bold').parent.parent.findNext('td').contents[0] if agent else None
        print(agent)
        
        current_club = soup.find_next('span', class_='info-table__content--bold').parent.parent.findNext('td').contents[0] if current_club else None
        print(current_club)
        
        joined = soup.find_next('span', class_='info-table__content--bold').parent.parent.findNext('td').contents[0] if joined else None
        print(joined)
        
        contract_expiry = soup.find_next('span', class_='info-table__content--bold').parent.parent.findNext('td').contents[0] if contract_expiry else None
        print(contract_expiry)

        player_bio_dict = create_dict(player_name, DOB, POB, age, height, citizenship, position, foot, agent, current_club, joined, contract_expiry)
        player_bio_list.append(player_bio_dict)

    

https://www.transfermarkt.co.uk/caxala/profil/spieler/886645
<div class="info-table info-table--right-space">
 <span class="info-table__content info-table__content--regular">
  Full name:
 </span>
 <span class="info-table__content info-table__content--bold">
  ? Caxala
 </span>
 <span class="info-table__content info-table__content--regular">
  Citizenship:
 </span>
 <span class="info-table__content info-table__content--bold">
  <img alt="Angola" class="flaggenrahmen" src="https://tmssl.akamaized.net/images/flagge/tiny/6.png?lm=1520611569" title="Angola"/>
  Angola
 </span>
 <span class="info-table__content info-table__content--regular">
  Position:
 </span>
 <span class="info-table__content info-table__content--bold">
  Defender
 </span>
 <span class="info-table__content info-table__content--regular">
  Current club:
 </span>
 <span class="info-table__content info-table__content--bold info-table__content--flex">
  <a href="/santa-rita-de-cassia-fc/startseite/verein/">
   <img alt="Sant

KeyboardInterrupt: 

In [33]:
print(player_bio_list)

[]


In [27]:
player_bio_list = []

for player in list_player_url:
    print(player)
    r = requests.get(url=player, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    player_name = soup.select('h1')[0].get_text().lower()
    print(player_name)

    DOB = None
    POB = None
    COB = None
    position = None
    age = None
    height = None
    foot = None
    citizenship = None
    second_citizenship = None
    agent = None
    current_club = None
    joined = None
    contract_expiry = None


    for row in soup.select('tr'):
        try:

            if row.select('th')[0].get_text().strip() == "Date of birth:":
                DOB = row.select('td')[0].get_text().strip()

            # COB = None
            if row.select('th')[0].get_text().strip() == "Place of birth:":
                POB = row.select('td')[0].get_text().strip()
                COB = row.select('td')[0].select('img')[0]['alt']
                
            if row.select('th')[0].get_text().strip() == "Position:":
                position = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Age:":
                age = int(row.select('td')[0].get_text().strip())

            if row.select('th')[0].get_text().strip() == "Height:":
                height = int(float(row.select('td')[0].get_text().strip().replace('m', '').replace(',', '.').strip())*100)

            if row.select('th')[0].get_text().strip() == "Foot:":
                foot = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Agent:":
                foot = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Current club:":
                POB = row.select('td')[0].get_text().strip()
                COB = row.select('td')[0].select('img')[0]['alt']

            if row.select('th')[0].get_text().strip() == "Joined":
                foot = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Contract expires:":
                foot = row.select('td')[0].get_text().strip()

        except:
            pass

    if COB == None:
        for row in soup.select('tr'):
            try:
                if row.select('th')[0].get_text().strip() == "Citizenship:":
                    COB = row.select('td')[0].get_text().strip()
            except:
                pass

    for row in soup.select('tr'):
        try:
            if row.select('th')[0].get_text().strip() == "Citizenship:":
                no_of_citizenships = len(row.select('td')[0].select('img'))
                if no_of_citizenships < 2:
                    citizenship = row.select('td')[0].select('img')[0]['alt']
                if no_of_citizenships > 1:
                    second_citizenship = row.select('td')[0].select('img')[1]['alt']
                else:
                    second_citizenship = None
        except:
            pass

        player_bio_dict = create_dict(player_name, DOB, POB, age, height, citizenship, position, foot, agent, current_club, joined, contract_expiry)
        player_bio_list.append(player_bio_dict)

https://www.transfermarkt.co.uk/caxala/profil/spieler/886645


                        #26                    
caxala 
https://www.transfermarkt.co.uk/jano-velasco/profil/spieler/94096

                                jano velasco 
https://www.transfermarkt.co.uk/stiven-quinonez/profil/spieler/500075

                                stiven quiñónez 
https://www.transfermarkt.co.uk/denilson-duran/profil/spieler/981944


                        #4                    
                                denilson durán 
https://www.transfermarkt.co.uk/diego-diaz/profil/spieler/214814


                        #8                    
                                diego díaz 
https://www.transfermarkt.co.uk/gonzalo-montes/profil/spieler/227096


                        #8                    
                                gonzalo montes 
https://www.transfermarkt.co.uk/ekbal-hussain/profil/spieler/929986


                        #77                    
                                ekbal hu

KeyboardInterrupt: 

In [28]:
print(player_bio_list)

[{'Player Name': '\n\n                        #26                    \ncaxala ', 'Date of Birth': None, 'Place of Birth': None, 'Age': None, 'Height': None, 'Citizenship': None, 'Position': None, 'Foot': None, 'Agent': None, 'Current Club': None, 'Joined': None, 'Contract Expiry': None}, {'Player Name': '\n\n                        #26                    \ncaxala ', 'Date of Birth': None, 'Place of Birth': None, 'Age': None, 'Height': None, 'Citizenship': None, 'Position': None, 'Foot': None, 'Agent': None, 'Current Club': None, 'Joined': None, 'Contract Expiry': None}, {'Player Name': '\n\n                        #26                    \ncaxala ', 'Date of Birth': None, 'Place of Birth': None, 'Age': None, 'Height': None, 'Citizenship': None, 'Position': None, 'Foot': None, 'Agent': None, 'Current Club': None, 'Joined': None, 'Contract Expiry': None}, {'Player Name': '\n\n                        #26                    \ncaxala ', 'Date of Birth': None, 'Place of Birth': None, 'Age': N